In [1]:
from music21 import stream, interval, corpus, instrument
from music21 import converter, note, chord, environment, duration
import notebook
import argparse
import pandas as pd
import pathlib
import numpy as np
import seaborn as sns
import matplotlib as plt
import turtle as ttl
import random
verbose = 0

## Chaos Game references
### Wikipedia
[Iterated Function Systems](https://en.wikipedia.org/wiki/Iterated_function_system)

[Chaos Game](https://en.wikipedia.org/wiki/Chaos_game)

[Barnsley Fern](https://en.wikipedia.org/wiki/Barnsley_fern)

[L-System](https://en.wikipedia.org/wiki/L-system)

### Python Packages

[turtle graphics](https://docs.python.org/3/library/turtle.html)<br>
Python 3.10 lib/turtle.py documentation

[The Chaos Game](https://beltoforion.de/en/recreational_mathematics/chaos_game.php)<br>
An implementation of the Chaos Game using polygons

In [ ]:
import turtle as ttl
import random
draw = True
verbose = 0

#
# An example IFS (Barnsley Fern) drawn using turtle graphics
#
if draw:
    pen = ttl.Turtle()
    ttl.clearscreen()
    pen.speed(0)
    pen.color("green")
    pen.penup()

x = 0
y = 0
limit = 500   # set to 11000 for more detail
for n in range(limit):
    sx = 65 * x
    sy = 37 * y -252  # scale the fern to fit nicely inside the window
    if draw:
        pen.goto(sx, sy)
        pen.pendown()
        pen.dot(3)
        pen.penup()
    r = random.random()
    if verbose > 0:
        print('r={} \tx,y = {},{}, \tscaled = {},{}'.format(r,x,y,sx,sy))

    if r < 0.01:
        x, y =  0.00 * x + 0.00 * y,  0.00 * x + 0.16 * y + 0.00
    elif r < 0.86:
        x, y =  0.85 * x + 0.04 * y, -0.04 * x + 0.85 * y + 1.60
    elif r < 0.93:
        x, y =  0.20 * x - 0.26 * y,  0.23 * x + 0.22 * y + 1.60
    else:
        x, y = -0.15 * x + 0.28 * y,  0.26 * x + 0.24 * y + 0.44
        
print("Done")
ttl.exitonclick()
ttl.done()


Done


In [2]:
import pandas as pd

class LinearFunction(object):
    """Defines a linear function f(x,y) =  (ax + by + c, dx + ey + f)
        The function name is 'f1', 'f2', etc. and needs to be unique
        as a member of an iterated function system (IFS)
    """
    columns = ['a', 'b', 'c', 'd', 'e', 'f', 'p']
    round_places = 4
    
    def __init__(self, x_coeficients:(float), y_coeficients:(float), probability = 0.0):
        self.name = None
        self.probability = probability
        self.x_coeficients = x_coeficients   # a 3-tupple: a,b,c
        self.y_coeficients = y_coeficients   # a 3-tupple: d,e,f
        coefs = list(x_coeficients + y_coeficients)    # all the coeficients + the probability as a Series
        coefs.append(probability)
        self.coeficients = pd.Series(coefs)
        self.coeficients.index = LinearFunction.columns
    
    def __str__(self):
        if self.name is None:
            return f"f=({self.x_coeficients}), ({self.y_coeficients})"
        else:
            return f"{self.name}=({self.x_coeficients}), ({self.y_coeficients})"
    
    def evaluate_at(self, point):
        x = point[0]
        y = point[1]
        nx = self.x_coeficients[0]*x + self.x_coeficients[1]*y + self.x_coeficients[2]
        ny = self.y_coeficients[0]*x + self.y_coeficients[1]*y + self.y_coeficients[2]
        return (round(nx, LinearFunction.round_places), round(ny, LinearFunction.round_places))
        
        
class IFS(object):
    """Iterated Function System
    
        An IFS is a system of linear functions (techincally an affine transformation) 
        implemented as a DataFrame where each row defines a linear function:
        f(x,y) = (ax + by + c, dx + ey + d)
        Associated with each function is a probability p, such that the sum of
        all probabilities is 1. In the chaos game, this is the probability of
        that function being selected.
        
    """
    
    def __init__(self, lf=None, probability:float = 0.0):
        self.ifs_df = pd.DataFrame(columns=LinearFunction.columns)
        self.functions = []   # list of LinearFunction
        if lf is not None:
            self.add_function(lf, probability)
        
    def add_function(self, linear_function:LinearFunction, probability:float):
        num = len(self.ifs_df)
        linear_function.name = f"f{num}"
        self.functions.append(linear_function)
        coefs = linear_function.coeficients
        coefs['p'] = probability
        coefs['linear_function'] = linear_function
        coefs['name'] = linear_function.name
        self.ifs_df = self.ifs_df.append(coefs, ignore_index=True)
        
    def size(self):
        return len(self.ifs_df)
    
    def index(self, ind):
        return self.ifs_df.iloc[ind]['linear_function']
        

In [3]:
class PointScaler(object):
    """ Scales a (x,y) point
        Arguments:
            gridsize - the overall size as an integer ordered pair. Default is (800, 800)
            point_stats is a dict with keys 'minX', 'minY', 'maxX', 'maxY'
        The origin is the center of the grid, so the range of x-values is (-gridsize[0]/2, gridsize[0]/2)
        and y-values (-gridsize[1]/2, gridsize[1]/2)
    """
    
    def __init__(self, point_stats, gridsize = (800, 800)):
        self.xrange = (-gridsize[0]/2, gridsize[0]/2)
        self.yrange = (-gridsize[1]/2, gridsize[1]/2)
        self.point_stats = point_stats
        self.xspan = gridsize[0]
        self.yspan = gridsize[1]
        self.xPointSpan = point_stats['maxX'] - point_stats['minX']
        self.yPointSpan = point_stats['maxY'] - point_stats['minY']
    
    def scale(self, point):
        x = point[0]
        y = point[1]
        sx = ((x - self.point_stats['minX']) / self.xPointSpan * self.xspan ) + self.xrange[0]
        sy = ((y - self.point_stats['minY']) / self.yPointSpan * self.yspan ) + self.yrange[0]
        return round(sx),round(sy)

In [4]:
import turtle as ttl
import random

class ChaosGame(object):
    """ Create and run a Chaos Game.
    Pseudo code:
        (x, y), = a random point in the biunit square
        iterate {
            i = a random integer from 0 to n to 1 inclusive (randomly select the function)
            (x, y) = Fi(x,y)
            plot x y( ), except during the first 20 iterations
        }
    
    """
    
    def __init__(self, ifs:IFS, size={'rows':800, 'columns':800}, draw=False, verbose = 0, point_scaler = None, reps = 1):
        self.verbose = verbose
        self.ifs = ifs
        self.size = size
        self.pen = None
        self.prob_sums = ifs.ifs_df['p'].cumsum()
        self.draw = draw
        self.reps = reps   # number of times to repeat the game
        self.colors = ['red','green','blue','orange','yellow','pink']
        self.points_df = pd.DataFrame(columns=['point', 'x', 'y', 'scaled_point', 'scaleX', 'scaleY', 'count'])
        self.min_max_dict = None
        self.point_scaler = point_scaler    # a PointScaler instance
    
    def scale_point(self, point):
        """Scales a point to a specified size.
            The result is an integer point (tupple)
            
        """
        x = (point[0] + 1)/2 * self.size['columns']
        y = (point[1] + 1)/2 * self.size['rows']
        return x,y
    
    def select_function(self):
            #
            # select a linear function
            #
            r = random.random()
            x =self.prob_sums >= r
            lf_to_select = x.tolist().index(True)    # the index of the first True in the list
            linear_function = self.ifs.index(lf_to_select)
            return linear_function
    
    def point_range(self):
        mm = self.min_max()
        return round(abs(mm['maxX']-mm['minX']), 4), round(abs(mm['maxY']-mm['minY']),4)
        
    def min_max(self):
        if self.min_max_dict is None:
            self.min_max_dict = {'minX': self.points_df['x'].min(), 'minY':self.points_df['y'].min(),\
            'maxX': self.points_df['x'].max(), 'maxY':self.points_df['y'].max()}
        return self.min_max_dict
    
    def record_point(self, point, scaled_point):
        row = pd.Series(data={'point':point, 'x':point[0], 'y':point[1], 'scaled_point':scaled_point,\
              'scaleX':scaled_point[0], 'scaleY':scaled_point[1], 'count':1})
        self.points_df = self.points_df.append(row, ignore_index=True)
    
    def scale_point(self, point):
        if self.point_scaler is None:
            sx = round(point[0] * 65)
            sy = round(37 * point[1] - 252)
            return (sx, sy)
        else:
            return self.point_scaler.scale(point)
    
    def draw_points(self):
        pass
        
    def run(self, limit=1000):
        if self.draw:
            self.pen = ttl.Turtle()
            ttl.clearscreen()
            self.pen.speed(0)    # 0=the fastest, 10=fast, 6=normal, 3=slow, 1 = slowest
            self.pen.penup()

        for i in range(self.reps):
            if self.draw:
                self.pen.color(self.colors[i])
            point =  (random.uniform(-1,1),random.uniform(-1,1))  # pick a point in biunit square [-1, 1]
            count = 0
            for n in range(limit):
                #
                # select a linear function
                # 
                linear_function = self.select_function()
                point = linear_function.evaluate_at(point)
                if count >= 20:
                    #
                    # scale and draw the point
                    #
                    # sx = round(point[0] * 65)
                    # sy = round(37 * point[1] - 252)
                    sx,sy = self.scale_point(point)
                    if self.verbose > 0:
                        print('n={}, point={}\t sx,sy = {},{}'.format(n, point, sx, sy))
                    self.record_point(point, (sx, sy))
                    if self.draw:
                        self.pen.goto(sx, sy)
                        self.pen.pendown()
                        self.pen.dot(1)
                        self.pen.penup()                        
                count = count + 1
        if self.draw:
            print("Done")
            ttl.exitonclick()
            ttl.done()

In [5]:
f1 = LinearFunction((0.0, 0.0, 0.0), (0.16, 0, 0))
f2 = LinearFunction((0.85, 0.04, 0.0), (-0.04, 0.85, 1.6))
f3 = LinearFunction((0.20, -0.26, 0.0), (0.23, 0.22, 1.60))
f4 = LinearFunction((-0.15, -0.28, 0.0), (0.26, 0.24, 0.44))
print(str(f2))
ifs = IFS(f1, 0.01)
ifs.add_function(f2, 0.85)
ifs.add_function(f3, 0.07)
ifs.add_function(f4, 0.07)
ifs.ifs_df

f=((0.85, 0.04, 0.0)), ((-0.04, 0.85, 1.6))


,a,b,c,d,e,f,p,linear_function,name
0,0.00,0.00,0.0,0.16,0.00,0.00,0.01,"f0=((0.0, 0.0, 0.0)), ((0.16, 0, 0))",f0
1,0.85,0.04,0.0,-0.04,0.85,1.60,0.85,"f1=((0.85, 0.04, 0.0)), ((-0.04, 0.85, 1.6))",f1
2,0.20,-0.26,0.0,0.23,0.22,1.60,0.07,"f2=((0.2, -0.26, 0.0)), ((0.23, 0.22, 1.6))",f2
3,-0.15,-0.28,0.0,0.26,0.24,0.44,0.07,"f3=((-0.15, -0.28, 0.0)), ((0.26, 0.24, 0.44))",f3


In [6]:
prob_sums = ifs.ifs_df['p'].cumsum()
print(prob_sums)
r =  random.random()
print(r)
x =prob_sums >= r
lf_to_select = x.tolist().index(True)
print(f"execute function: {lf_to_select}")

0    0.01
1    0.86
2    0.93
3    1.00
Name: p, dtype: float64
0.897355988233175
execute function: 2


In [9]:
cg = ChaosGame(ifs, draw=False, verbose=0)
cg.run(limit = 1000)
print(cg.points_df.head())

               point       x       y scaled_point scaleX scaleY count
0  (-0.5815, 6.4915) -0.5815  6.4915   (-38, -12)    -38    -12     1
1   (-0.2346, 7.141) -0.2346  7.1410    (-15, 12)    -15     12     1
2   (0.0862, 7.6792)  0.0862  7.6792      (6, 32)      6     32     1
3   (0.3804, 8.1239)  0.3804  8.1239     (25, 49)     25     49     1
4   (0.6483, 8.4901)  0.6483  8.4901     (42, 62)     42     62     1


In [10]:
p = (0.194, 5.528)
# cg.points_df[['x','y']].isin([p[0], p[1]])
# x = cg.points_df.groupby('point').count()
print(cg.min_max())
print(cg.point_range())
print(cg.prob_sums)

{'minX': -3.1375, 'minY': -0.2327, 'maxX': 2.523, 'maxY': 9.9619}
(5.6605, 10.1946)
0    0.01
1    0.86
2    0.93
3    1.00
Name: p, dtype: float64


In [ ]:
point_stats = cg.min_max()
scaler = PointScaler(point_stats)
cg = ChaosGame(ifs, draw=True, verbose=0, point_scaler=scaler, reps=4)
cg.run(limit = 500)

In [35]:
# show convergence
point = (random.uniform(-1,1),random.uniform(-1,1))
linear_function = f2
for i in range(20):
    # print(point)
    point = linear_function.evaluate_at(point)

In [11]:
#
# Sierpinski
#
s1 = LinearFunction((0.5, 0.0, 0.0), (0, 0.5, 0.5))
s2 = LinearFunction((.5, 0, .5), (0, .5, .5))
s3 = LinearFunction((.5, 0, .25), (0, .5, 0))

sierpinski = IFS(s1, 0.3333)
sierpinski.add_function(s2, 0.3333)
sierpinski.add_function(s3, 0.3333)
sierpinski.ifs_df

,a,b,c,d,e,f,p,linear_function,name
0,0.5,0.0,0.00,0.0,0.5,0.5,0.3333,"f0=((0.5, 0.0, 0.0)), ((0, 0.5, 0.5))",f0
1,0.5,0.0,0.50,0.0,0.5,0.5,0.3333,"f1=((0.5, 0, 0.5)), ((0, 0.5, 0.5))",f1
2,0.5,0.0,0.25,0.0,0.5,0.0,0.3333,"f2=((0.5, 0, 0.25)), ((0, 0.5, 0))",f2


In [37]:
# show convergence
point = (random.uniform(-1,1),random.uniform(-1,1))
linear_function = s3
for i in range(20):
    print(point)
    point = linear_function.evaluate_at(point)

(-0.3769938143248692, 0.6584844244716632)
(0.0615, 0.3292)
(0.2807, 0.1646)
(0.3903, 0.0823)
(0.4451, 0.0411)
(0.4726, 0.0205)
(0.4863, 0.0103)
(0.4931, 0.0052)
(0.4965, 0.0026)
(0.4982, 0.0013)
(0.4991, 0.0006)
(0.4995, 0.0003)
(0.4998, 0.0001)
(0.4999, 0.0001)
(0.5, 0.0001)
(0.5, 0.0001)
(0.5, 0.0001)
(0.5, 0.0001)
(0.5, 0.0001)
(0.5, 0.0001)


In [12]:
cg2 = ChaosGame(sierpinski, draw=False, verbose = 0)
cg2.run(limit = 1000)
print(cg2.points_df.head())
print(cg2.min_max())
print(cg2.point_range())

              point       x       y scaled_point scaleX scaleY count
0  (0.1007, 0.8422)  0.1007  0.8422    (7, -221)      7   -221     1
1  (0.3004, 0.4211)  0.3004  0.4211   (20, -236)     20   -236     1
2  (0.4002, 0.2105)  0.4002  0.2105   (26, -244)     26   -244     1
3  (0.2001, 0.6052)  0.2001  0.6052   (13, -230)     13   -230     1
4    (0.35, 0.3026)  0.3500  0.3026   (23, -241)     23   -241     1
{'minX': 0.0134, 'minY': 0.0154, 'maxX': 0.9933, 'maxY': 0.9999}
(0.9799, 0.9845)


In [13]:
point_stats = cg2.min_max()
scaler = PointScaler(point_stats, gridsize=(300,300))
cg2 = ChaosGame(sierpinski, draw=True, verbose=0, point_scaler=scaler)
cg2.run(limit = 10000)

Done


Terminator: 

In [17]:
cg2.points_df

,point,x,y,scaled_point,scaleX,scaleY,count
0,"(0.697, 0.4889)",0.6970,0.4889,"(76, -6)",76,-6,1
1,"(0.5985, 0.2445)",0.5985,0.2445,"(35, -105)",35,-105,1
2,"(0.5493, 0.1222)",0.5493,0.1222,"(15, -155)",15,-155,1
3,"(0.2747, 0.5611)",0.2747,0.5611,"(-98, 23)",-98,23,1
4,"(0.1373, 0.7806)",0.1373,0.7806,"(-155, 111)",-155,111,1
...,...,...,...,...,...,...,...
1975,"(0.5871, 0.9586)",0.5871,0.9586,"(31, 183)",31,183,1
1976,"(0.7935, 0.9793)",0.7935,0.9793,"(116, 192)",116,192,1
1977,"(0.6467, 0.4896)",0.6467,0.4896,"(55, -6)",55,-6,1
1978,"(0.5734, 0.2448)",0.5734,0.2448,"(25, -105)",25,-105,1


In [19]:
gb = cg2.points_df[['scaled_point', 'scaleX', 'scaleY']].groupby('scaled_point').count()
gb[gb['scaleX'] > 1]

,scaleX,scaleY
scaled_point,,
"(-184, 149)",2,2
"(-181, 192)",2,2
"(-177, 174)",3,3
"(-166, 193)",2,2
"(-155, 143)",2,2
...,...,...
"(112, 34)",2,2
"(135, 198)",2,2
"(138, 95)",2,2


In [42]:
df = pd.DataFrame(columns=['point','x','y'])
point = (random.uniform(-1,1),random.uniform(-1,1))
l = pd.Series(data={'point':point, 'x':point[0], 'y':point[1]})
df.append(l, ignore_index=True)

,point,x,y
0,"(0.686643069442989, -0.8763333390683676)",0.686643,-0.876333


In [40]:
#
# draw outline of region size 800 x 800. (0,0) is center of screen
# top right = (400, 400), bottom left = (-400, -400)
#
pos1 = ttl.position()
print(pos1)
ttl.color('red', 'green')
ttl.circle(10)
ttl.forward(400)
ttl.left(90)
ttl.forward(400)
ttl.left(90)
ttl.forward(400)
ttl.circle(10)
ttl.left(90)
ttl.forward(800)
ttl.dot(5, 'blue')
ttl.right(90)
ttl.forward(400)
ttl.dot(5, 'blue')
pos2 = ttl.position()
print(pos2)
ttl.exitonclick()
ttl.done()

(0.00,0.00)
(-400.00,-400.00)


Terminator: 